<a href="https://colab.research.google.com/github/NaotoMano1031/NaotoMano1031/blob/main/DataProcessingGRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GAIT_Sample作成のための床反力データの抽出

事前準備:reportというフォルダに必要なファイルが全て入っているようにする

参考サイト:https://note.nkmk.me/



Step 1 ファイルの抽出


In [23]:
import glob
import os
import pandas as pd

#読み込みたいフォルダのディレクトリを指定
excel_files = glob.glob('/content/report/*.xlsx')
print(excel_files)

#読み込みたいシート名を選択
for b in excel_files:
    excel_df = pd.read_excel(b,sheet_name='GSD')#明示的に
    excel_df.to_csv(f'{b}.csv',header=False) 

    
files = glob.glob('report/*.csv')
print(files)

['/content/report/1-6.xlsx']
['report/1-6.xlsx.csv']


Step 2 必要な値の計算と抽出

In [123]:

data_list = []
left_list = []
right_list = []

for a in files:
    df = pd.read_csv(a)
    df['filename'] = os.path.basename(a)
    data_list.append(df)
    print(df)
    
    #Braking and AP impulse算出用(立脚期全体を抽出)
    #1行目が揃っていないデータがあればエラーが出ます
    left = pd.DataFrame(df.loc[0:100, ['nt(%)','左Fz(Ave)','左Fx(Ave)']]).astype('float64')\
    .rename(columns= {'nt(%)': 'GC','左Fz(Ave)':'LtFz','左Fx(Ave)':'LtFx'})

    right = pd.DataFrame(df.loc[103:203, ['nt(%)','右Fz(Ave)','右Fx(Ave)']]).astype('float64')\
    .rename(columns= {'nt(%)': 'GC','右Fz(Ave)':'RtFz','右Fx(Ave)':'RtFx'})
    
    left = left.query('-1 < GC < 101 ')
    right = right.query('-1 < GC < 101 ')
    
    #Mean and Peak Propulsion算出用(立脚期0-20%を除去)
    left2 = left.query('19 < GC < 101 ').rename(columns= {'LtFz':'LtFz2','LtFx':'LtFx2'})
    right2 = right.query('19 < GC < 101 ').rename(columns= {'RtFz':'RtFz2','RtFx':'RtFx2'})

    #Mean Fz,Peak Braking,Fzを算出
    left3 = left.agg(['max','sum','mean','min']).drop('GC',axis = 1)
    right3 = right.agg(['max','sum','mean','min']).drop('GC',axis = 1)
    print(left3)    
    print(right3)
    #AP-impulse,Mean Peak propulsion,PPratioを算出
    
    left4 = left2.query(' 0 < LtFx2').agg(['max','sum','mean']).drop(['GC','LtFz2'],axis = 1)
    right4 = right2.query(' 0 < RtFx2').agg(['max','sum','mean']).drop(['GC','RtFz2'],axis = 1)
    print(left4)
    print(right4)
    

    
    
    #left_list.append(left)
    #right_list.append(right)

       0 nt(%) 総合Fz(Ave) 総合Fz(SD)  ... 右(sd) 総合(ave) 総合(sd)      filename
0      1     0   61.6515  36.2303  ...   NaN     NaN    NaN  1-6.xlsx.csv
1      2     1   65.4648  34.9689  ...  9.94   37.06   7.78  1-6.xlsx.csv
2      3     2   68.2578  33.2612  ...  5.67   35.29  10.78  1-6.xlsx.csv
3      4     3   70.8327   31.884  ...  1.94    71.8   7.97  1-6.xlsx.csv
4      5     4    73.952  31.6633  ...  1.61   62.82   1.74  1-6.xlsx.csv
..   ...   ...       ...      ...  ...   ...     ...    ...           ...
199  200    96   81.6405  38.2583  ...   NaN     NaN    NaN  1-6.xlsx.csv
200  201    97     79.47  38.1096  ...   NaN     NaN    NaN  1-6.xlsx.csv
201  202    98   77.4116  37.9229  ...   NaN     NaN    NaN  1-6.xlsx.csv
202  203    99   75.4596  37.8272  ...   NaN     NaN    NaN  1-6.xlsx.csv
203  204   100   73.7072  37.9122  ...   NaN     NaN    NaN  1-6.xlsx.csv

[204 rows x 29 columns]
             LtFz       LtFx
max    102.139100   7.105100
sum   8044.495100  65.236500


[       0 nt(%) 総合Fz(Ave) 総合Fz(SD)  ... 右(sd) 総合(ave) 総合(sd)      filename
0      1     0   61.6515  36.2303  ...   NaN     NaN    NaN  1-6.xlsx.csv
1      2     1   65.4648  34.9689  ...  9.94   37.06   7.78  1-6.xlsx.csv
2      3     2   68.2578  33.2612  ...  5.67   35.29  10.78  1-6.xlsx.csv
3      4     3   70.8327   31.884  ...  1.94    71.8   7.97  1-6.xlsx.csv
4      5     4    73.952  31.6633  ...  1.61   62.82   1.74  1-6.xlsx.csv
..   ...   ...       ...      ...  ...   ...     ...    ...           ...
199  200    96   81.6405  38.2583  ...   NaN     NaN    NaN  1-6.xlsx.csv
200  201    97     79.47  38.1096  ...   NaN     NaN    NaN  1-6.xlsx.csv
201  202    98   77.4116  37.9229  ...   NaN     NaN    NaN  1-6.xlsx.csv
202  203    99   75.4596  37.8272  ...   NaN     NaN    NaN  1-6.xlsx.csv
203  204   100   73.7072  37.9122  ...   NaN     NaN    NaN  1-6.xlsx.csv

[204 rows x 29 columns]]


In [ ]:

#listを列方向に結合（axis=1）    
left_df = pd.concat(left_list, axis=1)
right_df = pd.concat(right_list, axis=1)

left_df2 = left_df[['value']]
right_df2 = right_df[['value']]

name = left_df2.tail(1)

name.to_csv("name.csv")

left_df3 = left_df2.head(1200)
right_df3 = right_df2.head(1200)

left_df4 = left_df3.astype('float64')
right_df4 = right_df3.astype('float64')

subtraction =left_df4.sub(right_df4, axis='index')
mean = (left_df4 + right_df4)/2

left_df4.to_csv("left.csv")
right_df4.to_csv("right.csv")
subtraction.to_csv("subtraction.csv")
mean.to_csv("mean.csv")


df2 = pd.concat([left_df4, right_df4, subtraction, mean])
df3 = df2[['value']]
df3.to_csv("result.csv") 